In [ ]:
import numpy as np
import noise

from plotly import graph_objs as plgo
from plotly import offline as ploff
ploff.init_notebook_mode(connected=False)

In [ ]:
base = np.linspace(0, 5, 500)
x, y = np.meshgrid(base, base)
final_func = np.vectorize(lambda x, y: noise.pnoise2(x, y, octaves=4))
land = final_func(x, y)

water = np.zeros(land.shape)
water[240:261,240:261] = 0.5 - land[240:261,240:261]

In [ ]:
def plot(x, y, land, water):
    mask = ~water.astype(bool)
    layer = land + water
    layer[mask] = None
    ploff.iplot(
        plgo.Figure({
            'data': [
                plgo.Surface({
                    'name': 'land',
                    'x': x,
                    'y': y,
                    'z': land,
                    'colorscale': [(0.0, '#F9EBEA'), (1.0, '#641E16')],
                }),
                plgo.Surface({
                    'name': 'water',
                    'x': x,
                    'y': y,
                    'z': layer,
                    'colorscale': [(0.0, '#154360'), (1.0, '#85C1E9')],
                    'showscale': False,
                })
            ],
            'layout': plgo.Layout({
                'title': 'Perlin Noise',
                'width': 800,
                'height': 800,
                'scene': plgo.Scene({
                    'zaxis': plgo.ZAxis({
                        'range': [-1, 1.5]
                    })
                })
            }),
        })
    )

In [ ]:
def simple_automata_step(terra,water,alpha):
    assert(terra.shape==water.shape)
    levels = terra+water
    # Get the level transfer:
    transfer_x = levels[:,:-1]-levels[:,1:]
    transfer_x = np.minimum(water[:,:-1],transfer_x)
    transfer_x = np.maximum(-water[:,1:],transfer_x)
    #
    transfer_y = levels[:-1,:]-levels[1:,:]
    transfer_y = np.minimum(water[:-1,:],transfer_y)
    transfer_y = np.maximum(-water[1:,:],transfer_y)
    # Update water levels:
    nwater = water.copy()
    nwater[:,:-1] -= transfer_x*alpha
    nwater[:,1:]  += transfer_x*alpha
    nwater[:-1,:] -= transfer_y*alpha
    nwater[1:,:]  += transfer_y*alpha
    return nwater

In [ ]:
w = simple_automata_step(land, water, 0.1)

In [ ]:
for i in range(100):
    w = simple_automata_step(land, w, 0.1)